In [1]:
# move to source directory
%pwd
%cd ../src
%pwd
%ls

/home/yh/git/kaggle_ieee-fraud-detection/src
__pycache__/    feature_factory.py*   models/              utils/
config/         features/             tests/               visualization/
configure.py*   main.py*              trainer_factory.py*
experiment.py*  modelapi_factory.py*  transformer.py*


In [2]:
# major modules
import pandas as pd
import numpy as np
import sys
sys.stdout.flush()

In [3]:
        # COLUMNS WITH STRINGS
        str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5',
                    'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
                    'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

        # FIRST 53 COLUMNS
        cols = ['TransactionID', 'TransactionDT', 'TransactionAmt',
                'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
                'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
                'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
                'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
                'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
                'M5', 'M6', 'M7', 'M8', 'M9']

        # V COLUMNS TO LOAD DECIDED BY CORRELATION EDA
        # https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id
        v = [1, 3, 4, 6, 8, 11]
        v += [13, 14, 17, 20, 23, 26, 27, 30]
        v += [36, 37, 40, 41, 44, 47, 48]
        v += [54, 56, 59, 62, 65, 67, 68, 70]
        v += [76, 78, 80, 82, 86, 88, 89, 91]

        # v += [96, 98, 99, 104] #relates to groups, no NAN
        v += [107, 108, 111, 115, 117, 120, 121, 123]  # maybe group, no NAN
        v += [124, 127, 129, 130, 136]  # relates to groups, no NAN

        # LOTS OF NAN BELOW
        v += [138, 139, 142, 147, 156, 162]  # b1
        v += [165, 160, 166]  # b1
        v += [178, 176, 173, 182]  # b2
        v += [187, 203, 205, 207, 215]  # b2
        v += [169, 171, 175, 180, 185, 188, 198, 210, 209]  # b2
        v += [218, 223, 224, 226, 228, 229, 235]  # b3
        v += [240, 258, 257, 253, 252, 260, 261]  # b3
        v += [264, 266, 267, 274, 277]  # b3
        v += [220, 221, 234, 238, 250, 271]  # b3

        v += [294, 284, 285, 286, 291, 297]  # relates to grous, no NAN
        v += [303, 305, 307, 309, 310, 320]  # relates to groups, no NAN
        v += [281, 283, 289, 296, 301, 314]  # relates to groups, no NAN
        # v += [332, 325, 335, 338] # b4 lots NAN

        cols += ['V'+str(x) for x in v]
        dtypes = {}
        for c in cols+['id_0'+str(x) for x in range(1, 10)]+['id_'+str(x) for x in range(10, 34)]:
            dtypes[c] = 'float32'
        for c in str_type:
            dtypes[c] = 'category'

In [4]:
train = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/raw_train.pkl')
test = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/raw_test.pkl')
#train = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/nroman_train.pkl')
#test = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/nroman_test.pkl')
#train = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/features_train.pkl')
#test = pd.read_pickle('/home/yh/git/kaggle_ieee-fraud-detection/data/processed/features_test.pkl')

In [5]:
train = train.astype(dtypes)[cols]
test = test.astype(dtypes)[cols]

In [6]:
X_train = pd.read_csv('../data/raw/train_transaction.csv',
                      index_col='TransactionID', dtype=dtypes, usecols=cols+['isFraud'])
train_id = pd.read_csv('../data/raw/train_identity.csv',
                       index_col='TransactionID', dtype=dtypes)
X_train = X_train.merge(train_id, how='left', left_index=True, right_index=True)
# LOAD TEST
X_test = pd.read_csv('../data/raw/test_transaction.csv',
                     index_col='TransactionID', dtype=dtypes, usecols=cols)
test_id = pd.read_csv('../data/raw/test_identity.csv',
                      index_col='TransactionID', dtype=dtypes)
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)


In [7]:
print(train.shape, test.shape)

(590540, 174) (506691, 174)


In [8]:
print(X_train.shape, X_test.shape)

(590540, 214) (506691, 213)


In [32]:
print(len(cols))

174


In [18]:
print('train-X_train')
for col in train.columns:
    if col not in X_train.columns:
        print(col)

print('X_train-train')
for col in X_train.columns:
    if col not in train.columns:
        print(col)

train-X_train
TransactionID
V2
V5
V7
V9
V10
V12
V15
V16
V18
V19
V21
V22
V24
V25
V28
V29
V31
V32
V33
V34
V35
V38
V39
V42
V43
V45
V46
V49
V50
V51
V52
V53
V55
V57
V58
V60
V61
V63
V64
V66
V69
V71
V72
V73
V74
V75
V77
V79
V81
V83
V84
V85
V87
V90
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V109
V110
V112
V113
V114
V116
V118
V119
V122
V125
V126
V128
V131
V132
V133
V134
V135
V137
V140
V141
V143
V144
V145
V146
V148
V149
V150
V151
V152
V153
V154
V155
V157
V158
V159
V161
V163
V164
V167
V168
V170
V172
V174
V177
V179
V181
V183
V184
V186
V189
V190
V191
V192
V193
V194
V195
V196
V197
V199
V200
V201
V202
V204
V206
V208
V211
V212
V213
V214
V216
V217
V219
V222
V225
V227
V230
V231
V232
V233
V236
V237
V239
V241
V242
V243
V244
V245
V246
V247
V248
V249
V251
V254
V255
V256
V259
V262
V263
V265
V268
V269
V270
V272
V273
V275
V276
V278
V279
V280
V282
V287
V288
V290
V292
V293
V295
V298
V299
V300
V302
V304
V306
V308
V311
V312
V313
V315
V316
V317
V318
V319
V321
V322
V323
V324
V325
V326
V327
V32

In [14]:
for col in test.columns:
    if col not in train.columns:
        print(col)

id-01
id-02
id-03
id-04
id-05
id-06
id-07
id-08
id-09


In [9]:
test_identity = pd.read_csv(f'../data/raw/test_identity.csv')
for col in test_identity:
    print(col)
columns = {}
for i in range(1, 39):
    columns[f'id-f{i}'] = f'id_{i}'
for k, v in columns.items():
    print(k, v)

test_identity = test_identity.rename(columns=columns)

for col in test_identity:
    print(col)


TransactionID
id-01
id-02
id-03
id-04
id-05
id-06
id-07
id-08
id-09
id-10
id-11
id-12
id-13
id-14
id-15
id-16
id-17
id-18
id-19
id-20
id-21
id-22
id-23
id-24
id-25
id-26
id-27
id-28
id-29
id-30
id-31
id-32
id-33
id-34
id-35
id-36
id-37
id-38
DeviceType
DeviceInfo
id-1 id_1
id-2 id_2
id-3 id_3
id-4 id_4
id-5 id_5
id-6 id_6
id-7 id_7
id-8 id_8
id-9 id_9
id-10 id_10
id-11 id_11
id-12 id_12
id-13 id_13
id-14 id_14
id-15 id_15
id-16 id_16
id-17 id_17
id-18 id_18
id-19 id_19
id-20 id_20
id-21 id_21
id-22 id_22
id-23 id_23
id-24 id_24
id-25 id_25
id-26 id_26
id-27 id_27
id-28 id_28
id-29 id_29
id-30 id_30
id-31 id_31
id-32 id_32
id-33 id_33
id-34 id_34
id-35 id_35
id-36 id_36
id-37 id_37
id-38 id_38
TransactionID
id-01
id-02
id-03
id-04
id-05
id-06
id-07
id-08
id-09
id_10
id_11
id_12
id_13
id_14
id_15
id_16
id_17
id_18
id_19
id_20
id_21
id_22
id_23
id_24
id_25
id_26
id_27
id_28
id_29
id_30
id_31
id_32
id_33
id_34
id_35
id_36
id_37
id_38
DeviceType
DeviceInfo
